### Programming in life sciences

### Predicting open/close conformation of a kinase model



<b>Lecture 1: Introduction to alignments - exercise: align two homologous kinases (given as fasta files)</b>

Lecture 2: Introduction to superpositions - exercise: get RMSD of two structures

Lecture 3: Introduction to AlphaFold - exercise: get models from AlphaFold and classify them as open/close, do the same for models from Swiss-Model and AlphaFold-dropout

Lecture 4: Ligand docking (DiffDock) - exercise: get docking from DiffDock colab, find interacting atoms and visualize docking

### 1. Lecture: Needleman-Wunsch algorithm

In this exercise, we want to align two sequences obtained from fasta files

In [10]:
# numpy is a library providing a lot of numerical mathematical functionalities
# here we only use it to store stuff in matrices
import numpy as np

# we create a new class that will be used to define substitution matrices
class SubstitutionMatrix:
  """Read, store and analyse substitution matrices, default = BLOSUM62"""

  # this is the constructor function! When you create a new instance of type
  # Substitution Matrix, this function gets called. By default it reads in
  # the provided file: blosum62.txt... you can create your own matrix if
  # you like ;)
  def __init__(self, data_file: str = "blosum62.txt"):
    """ Constructor function for the SubstitutionMatrix class
    Parameters
    ----------
    data_file : string
        Filename of the substitution matrix to read in, must contain 21 lines,
        with the first line containing the 20 amino acid one letter codes,
        by default reads the blosum62.txt matrix in the current folder
    """
    
    # read all the lines from the file at once  ('r' = read)
    with open(data_file,'r') as fh:
      data = fh.readlines()

    # the size of subsitution matrices must be 20x20
    # the first line of the file gives the amino acid one letter codes
    # which gives a total of 21 lines in the file
    if len(data) != 21:
      err = "Expected exactly 21 lines in matrix file!"
      err += " One line defining the amino acid one letter codes and "
      err += "20 lines defining the substitution matrix"
      raise RuntimeError(err)    

    # get the amino acid one letter codes
    self.one_letter_codes = data[0].strip()

    # check that 20 amino acids are defined
    if len(self.one_letter_codes) != 20:
      err = "First line must contain exactly 20 letters representing "
      err += "the amino acids!"
      raise RuntimeError(err)

    # initialize the substitution matrix with zeros
    self.matrix = np.zeros((20,20))

    # fill in the matrix by reading the file line by line
    # note that range returns a vector starting at 0 to the integer in argument - 1 
    for i in range(20):
      data_line = data[1+i].split()
      if len(data_line) != 20:
        raise RuntimeError("Each data line must contain exactly 20 elements!")
      for j in range(20):
        self.matrix[(i,j)] = int(data_line[j])


  # The function where you can extract the score for a specific amino acid substitution...
  # e.g. mat.get_score('W', 'A')
  def get_score(self, aa_one: str, aa_two: str):
    """ Extract the substitution score for two amino acids from the substitution matrix
    Parameters
    ----------
    aa_one : string
        One letter amino acid code
    aa_two : string
        One letter amino acid code
    Returns
    -------
    self.matrix[(idx_one,idx_two)] : float
        Substitution matrix entry for the substitution of amino acid one and amino acid two,
        note that the matrices are symmetric, so get_score(aa_one, aa_two) is equal to get_score(aa_two, aa_one)
    """

    # get the indices of the amino acids in the matrix
    idx_one = self.one_letter_codes.find(aa_one)
    idx_two = self.one_letter_codes.find(aa_two)

    if idx_one == -1:
      raise RuntimeError("aa_one must be one of: " + self.one_letter_codes)
    
    if idx_two == -1:
      raise RuntimeError("aa_two must be one of: " + self.one_letter_codes)

    return self.matrix[(idx_one,idx_two)]


In [ ]:
# Decorating functions with docstrings is not only best practice,
# it also helps automated documentation generation with tools
# like Sphinx (https://www.sphinx-doc.org) or simply get more
# information using the Python help function:
help(SubstitutionMatrix.get_score)

In [ ]:
# Construct a default substitution matrix
subst_matrix = SubstitutionMatrix()

# get score of substituting ASP with ASN and GLY with PRO
print("Score D->N:", subst_matrix.get_score('N', 'D'))
print("Score G->P:", subst_matrix.get_score('G', 'P'))

In [17]:
def ReadFasta(filename: str):
    """ A simplistic reader that only checks if the first line starts with '>' and assumes that the whole rest is the input sequence
    Parameters
    ----------
    filename : string
        Filename of the FASTA file to read
    Returns
    -------
    s : string
        Sequence read from the input FASTA file
    """
    
    # store all the lines in a single list  ('r' = read)
    with open(filename, 'r') as fh:
        lines = fh.readlines()
    
    # handle errors
    if len(lines) <= 1:
        raise RuntimeError("Expect multiple lines in FASTA file")
    if not lines[0].startswith('>'):
        raise RuntimeError("Assume first line in FASTA file to start with '>'")

    # read the sequence lines, while removing all extra characters
    s = ""
    for line in lines[1:]:
        s += line.strip()
        
    return s

In [ ]:
# read in the two sequences to align 
# the two files are located in the same directory as the notebook
s1 = ReadFasta("s1.fasta")
s2 = ReadFasta("s2.fasta")
print(s1)
print(s2)

## old s1: GAWGHEE, old s2: PAWHEA

#### Write the code for running the Needleman-Wunsch algorithm

In [ ]:
# define a penalty for introducing a gap in the sequence alignment
gap_penalty = -8

Setup scoring and backtracking matrix

In [ ]:
n_rows = len(s1)+1 # s1 goes from top to bottom
n_cols = len(s2)+1 # s2 goes from left to right

scoring_matrix = np.zeros((n_rows, n_cols))

# backtrack encoding:
# 1 => aligned
# 2 => deletion in s1 (insertion in s2 respectively)
# 3 => deletion in s2 (insertion in s1 respectively)
backtrack_matrix = np.zeros((n_rows, n_cols))

# the first row and the first column can already be prefilled
for i in range(1, n_rows):
    scoring_matrix[(i,0)] = i*gap_penalty
    backtrack_matrix[(i,0)] = 3
for i in range(1, n_cols):
    scoring_matrix[(0,i)] = i*gap_penalty
    backtrack_matrix[(0,i)] = 2

print("row sequence: ", s1)
print("col sequence: ", s2)
print("empty scoring matrix: ")
print(scoring_matrix)
print("empty backtracking matrix: ")
print(backtrack_matrix)

#### Fill scoring and backtracking matrices

Every position in the scoring matrix represents the best local solution given the
steps I can take
- Align two residues => Move along the diagonal (consume a letter from s1 and s2),
                        the value of the scoring matrix to the upper left plus the corresponding score from the scoring matrix
- Apply deletion/gap in s1 => Move right in the scoring matrix (consume a letter from s2), 
                              the value from the scoring matrix to the left plus the penalty for a gap
- Apply deletion/gap in s2 => Move down in the scoring matrix (consume a letter from s1),
                              the value from the scoring matrix above plus the penalty for a gap

The goal is to identify the best scoring solution from those three,
set the according score in the scoring matrix and keep track of this local
step in the backtracking matrix. 

The backtracking matrix stores the path I took, e.g. 1 for the first option
(see backtracking encoding in previous code cell)

In [ ]:
for r_idx in range(1, n_rows):
    for c_idx in range(1, n_cols):
        score_match = scoring_matrix[(r_idx-1, c_idx-1)] + subst_matrix.get_score(s1[r_idx-1], s2[c_idx-1])
        score_del_1 = scoring_matrix[(r_idx, c_idx-1)] + gap_penalty
        score_del_2 = scoring_matrix[(r_idx-1, c_idx)] + gap_penalty        
        # TODO Fill the values at position (r_idx, c_idx) in scoring_matrix and backtracking_matrix
        # you can assign an element with
        # scoring_matrix[(r_idx, c_idx)] = my_awesome_value
        if score_match >= score_del_1 and score_match >= score_del_2:
            scoring_matrix[(r_idx, c_idx)] = score_match
            backtrack_matrix[(r_idx, c_idx)] = 1
        elif score_del_1 >= score_match and score_del_1 >= score_del_2:
            scoring_matrix[(r_idx, c_idx)] = score_del_1
            backtrack_matrix[(r_idx, c_idx)] = 2
        else:
            scoring_matrix[(r_idx, c_idx)] = score_del_2
            backtrack_matrix[(r_idx, c_idx)] = 3
        
print("filled scoring matrix:")
print(scoring_matrix)
print("filled backtracking matrix:")
print(backtrack_matrix)

#### Perform backtracking to get final alignment

Remember the encoding:

1 => aligned, move along the diagonal

2 => deletion in s1 (insertion in s2 respectively), move right

3 => deletion in s2 (insertion in s1 respectively), move down

In [ ]:
# In principle we start at the bottom right of the backtracking matrix and
# work our way through the matrix until we hit the upper left

r_idx = n_rows-1
c_idx = n_cols-1
path = list()

# if we hit zero, we're at the upper left corner and can stop
while backtrack_matrix[(r_idx, c_idx)] != 0:
    path.append(backtrack_matrix[(r_idx, c_idx)])
    if backtrack_matrix[(r_idx, c_idx)] == 1:
        r_idx -= 1
        c_idx -= 1
    elif backtrack_matrix[(r_idx, c_idx)] == 2:
        c_idx -= 1
    else:
        r_idx -= 1
        
# backtracking comes from the back, so lets reverse...
path.reverse()

aln_s1 = []
aln_s2 = []
s1_idx = 0
s2_idx = 0

for p in path:
    if p == 1:
        aln_s1.append(s1[s1_idx])
        aln_s2.append(s2[s2_idx])
        s1_idx += 1
        s2_idx += 1
    elif p == 2:
        aln_s1.append('-')
        aln_s2.append(s2[s2_idx])
        s2_idx += 1
    else:
        aln_s1.append(s1[s1_idx])
        aln_s2.append('-')
        s1_idx += 1
        
aln_s1 = ''.join(aln_s1)
aln_s2 = ''.join(aln_s2)
        
print("input sequence s1:", s1)
print("input sequence s2:", s2)
print("\noptimal alignment:")
print("s1:", aln_s1)
print("s2:", aln_s2)


In [15]:
#let's put everything in one function
def Align(s1: str, s2: str):
    """ Alignment of two strings of amino acids according to the Needleman-Wunsch algorithm
    Parameters
    ----------
    s1 : string
        First amino acid string to align
    s2 : string
        Second amino acid string to align
    Returns
    -------
    aln_s1 : string
        First amino acid string, aligned
    aln_s2 : string
        Second amino acid string, aligned
    """
    
    # initialize the substitution matrix (default = BLOSUM62)
    subst_matrix = SubstitutionMatrix()
    
    # define the gap penalty value
    gap_penalty = -8

    # initialize the scoring and backtrack matrices
    n_rows = len(s1)+1 # s1 goes from top to bottom
    n_cols = len(s2)+1 # s2 goes from left to right
    scoring_matrix = np.zeros((n_rows, n_cols))
    backtrack_matrix = np.zeros((n_rows, n_cols))

    # the first row and the first column can already be prefilled
    for i in range(1, n_rows):
        scoring_matrix[(i,0)] = i*gap_penalty
        backtrack_matrix[(i,0)] = 3
    for i in range(1, n_cols):
        scoring_matrix[(0,i)] = i*gap_penalty
        backtrack_matrix[(0,i)] = 2

    # fill the scoring and backtrack matrices
    for r_idx in range(1, n_rows):
        for c_idx in range(1, n_cols):
            score_match = scoring_matrix[(r_idx-1, c_idx-1)] + subst_matrix.get_score(s1[r_idx-1], s2[c_idx-1])
            score_del_1 = scoring_matrix[(r_idx, c_idx-1)] + gap_penalty
            score_del_2 = scoring_matrix[(r_idx-1, c_idx)] + gap_penalty        

            if score_match >= score_del_1 and score_match >= score_del_2:
                scoring_matrix[(r_idx, c_idx)] = score_match
                backtrack_matrix[(r_idx, c_idx)] = 1
            elif score_del_1 >= score_match and score_del_1 >= score_del_2:
                scoring_matrix[(r_idx, c_idx)] = score_del_1
                backtrack_matrix[(r_idx, c_idx)] = 2
            else:
                scoring_matrix[(r_idx, c_idx)] = score_del_2
                backtrack_matrix[(r_idx, c_idx)] = 3

    # track back from the lower right corner to the top left corner of the backtrack matrix to get the path
    r_idx = n_rows-1
    c_idx = n_cols-1
    path = list()

    # if we hit zero, we're at the upper left corner and can stop
    while backtrack_matrix[(r_idx, c_idx)] != 0:
        path.append(backtrack_matrix[(r_idx, c_idx)])
        if backtrack_matrix[(r_idx, c_idx)] == 1:
            r_idx -= 1
            c_idx -= 1
        elif backtrack_matrix[(r_idx, c_idx)] == 2:
            c_idx -= 1
        else:
            r_idx -= 1

    # backtracking comes from the back, so lets reverse...
    path.reverse()

    # get the aligned sequences by going through the path
    aln_s1 = []
    aln_s2 = []
    s1_idx = 0
    s2_idx = 0

    for p in path:
        if p == 1:
            aln_s1.append(s1[s1_idx])
            aln_s2.append(s2[s2_idx])
            s1_idx += 1
            s2_idx += 1
        elif p == 2:
            aln_s1.append('-')
            aln_s2.append(s2[s2_idx])
            s2_idx += 1
        else:
            aln_s1.append(s1[s1_idx])
            aln_s2.append('-')
            s1_idx += 1

    aln_s1 = ''.join(aln_s1)
    aln_s2 = ''.join(aln_s2)
    
    return aln_s1, aln_s2

In [18]:
# Read two fasta files of bacterial kinase homologs and get the alignment
bac1_kinase = ReadFasta('Q47XA8.fasta')
print(bac1_kinase)

bac2_kinase = ReadFasta('P69441.fasta')
print(bac2_kinase)


MRIVLLGAPGAGKGTQAQFLMAKFGIPQISTGDMLRAAIKAGSELGNKAKAVMDAGQLVSDDLIIGLVKERVAQEDCKAGFLLDGFPRTIPQADAMKESGIVVDHVLEFDVPDEVIVERMAGRRVHSGSGRVYHLVYNPPKVEGKDDVSGDDLSIRPDDEEATVRKRLAIYHEQTKPLVDFYQAEAKSGSCSYLTIDGTQAVEKVNQLLSAQLA
MRIILLGAPGAGKGTQAQFIMEKYGIPQISTGDMLRAAVKSGSELGKQAKDIMDAGKLVTDELVIALVKERIAQEDCRNGFLLDGFPRTIPQADAMKEAGINVDYVLEFDVPDELIVDRIVGRRVHAPSGRVYHVKFNPPKVEGKDDVTGEELTTRKDDQEETVRKRLVEYHQMTAPLIGYYSKEAEAGNTKYAKVDGTKPVAEVRADLEKILG


In [19]:
bac1_aln, bac2_aln = Align(bac1_kinase, bac2_kinase)
print(bac1_aln)
print(bac2_aln)

MRIVLLGAPGAGKGTQAQFLMAKFGIPQISTGDMLRAAIKAGSELGNKAKAVMDAGQLVSDDLIIGLVKERVAQEDCKAGFLLDGFPRTIPQADAMKESGIVVDHVLEFDVPDEVIVERMAGRRVHSGSGRVYHLVYNPPKVEGKDDVSGDDLSIRPDDEEATVRKRLAIYHEQTKPLVDFYQAEAKSGSCSYLTIDGTQAVEKVNQLLSAQLA
MRIILLGAPGAGKGTQAQFIMEKYGIPQISTGDMLRAAVKSGSELGKQAKDIMDAGKLVTDELVIALVKERIAQEDCRNGFLLDGFPRTIPQADAMKEAGINVDYVLEFDVPDELIVDRIVGRRVHAPSGRVYHVKFNPPKVEGKDDVTGEELTTRKDDQEETVRKRLVEYHQMTAPLIGYYSKEAEAGNTKYAKVDGTKPVAEVRADLEKILG


In [ ]:
# Sequence identity is a simple proxy for evolutionary distance
def SeqID(aln_s1: str, aln_s2: str):
    """ Computes sequence identity - the percentage of conserved
    amino acids
    ----------
    aln_s1 : string
        First aligned amino acid string
    aln_s2 : string
        Second aligned amino acid string
    Returns
    -------
    seqid : float
        The sequence identity
    """
    # must have same length
    assert(len(aln_s1) == len(aln_s2))
    n_match = 0
    n_aligned = 0
    for a,b in zip(aln_s1, aln_s2):
        if a!='-' and b!='-':
            n_aligned += 1
            if a == b:
                n_match += 1
    if n_aligned != 0:
        # explicitely cast to float here
        # This is not required in Python3 but Python2 would perform
        # an integer division otherwise
        return float(n_match)/n_aligned*100
    return 0.0

In [ ]:
# Python provides several ways of string formatting
# one example are so called f-strings
print(f"SeqID: {SeqID(bac1_aln, bac2_aln):.2f}%")

Lecture 1: Introduction to alignments - exercise: align two homologous kinases (given as fasta files)

<b>Lecture 2: Introduction to superpositions - exercise: get RMSD of two structures</b>

Lecture 3: Introduction to AlphaFold - exercise: get models from AlphaFold and classify them as open/close, do the same for models from Swiss-Model and AlphaFold-dropout

Lecture 4: Ligand docking (DiffDock) - exercise: get docking from DiffDock colab, find interacting atoms and visualize docking

### 2. Lecture: superpositions

In [1]:
# we create a new class that will be used to define atoms in 3D space
class Vec3:
    """Store and print 3D coordinates"""
    
    def __init__(self, x: float, y: float, z: float):
        """ Constructor function for the Vec3 class
        Parameters
        ----------
        x, y, z : float
            Coordinates of the atom in 3D space
        """

        # define member variables here
        self.x = x
        self.y = y
        self.z = z
        
    def __str__(self):
        """ Print function for the Vec3 class """
        return '[{:.2f}, {:.2f}, {:.2f}]'.format(self.x, self.y, self.z)

In [2]:
# we create a new class that will be used to describe a single residue of a protein chain
class Residue:
    """Store and print a protein residue"""
    def __init__(self, rname: str, rnum: int):
        """ Constructor function for the Residue class
        Parameters
        ----------
        rname : string
            Name of the residue (amino acid 3 letter code)
        rnum : int
            Residue sequence number, position in the chain
        """

        # define member variables here
        self.rname = rname
        self.rnum = rnum
        self.n_pos = None
        self.ca_pos = None
        self.c_pos = None
        
    def __str__(self):
        """ Print function for the Residue class """    
        pos_strings = list()
        pos_strings.append('N: ' + str(self.n_pos))
        pos_strings.append('CA: ' + str(self.ca_pos))
        pos_strings.append('C: ' + str(self.c_pos))
        return '\t'.join([self.rname, self.rnum, ' '.join(pos_strings)])
    
    def get_coords(self):
        """ Get all the coordinates of the atoms in the residue as a list
        Parameters
        ----------

        Returns
        -------
        list
            x, y and z Å coordinates of the N (amino group atom), CA (carbon atom) and C (carboxyl group atom) positions in a single list
        """
        return [self.n_pos.x, self.n_pos.y, self.n_pos.z,self.ca_pos.x, self.ca_pos.y, self.ca_pos.z,self.c_pos.x, self.c_pos.y, self.c_pos.z]


In [3]:
# define a Chain class, that contains following information as member variables:
# - name (e.g. "A")
# - a list of residues
#
#AWESOME IMPLEMENTATION COMES HERE:

class Chain:
    """Store and print a protein chain = a list of residues"""

    def __init__(self, cname: str):
        """ Constructor function for the Chain class
        Parameters
        ----------
        cname : string
            Name of the chain, e.g. 'A'
        """

        self.cname = cname
        self.residues = list()
        
    def __str__(self):
        """ Print function for the Chain class """    
        return_list = list()
        return_list.append('chain name: ' + self.cname)
        for r in self.residues:
            return_list.append(' ' + str(r))
        return '\n'.join(return_list)

# define an Assembly class, that contains following information as member variables:
# - a list of chains
#
# AWESOME IMPLEMENTATION COMES HERE:

class Assembly:
    """Store and print a protein assembly = a list of chains"""

    def __init__(self):
        """ Constructor function for the Assembly class
        Parameters
        ----------
        """

        self.chains = list()
        
    def __str__(self):
        """ Print function for the Assembly class """    
        return_list = list()
        for c in self.chains:
            return_list.append(str(c))
        return '\n\n'.join(return_list)

The PDB is a very specific file format for protein structures:
https://www.cgl.ucsf.edu/chimera/docs/UsersGuide/tutorials/pdbintro.html

In [4]:
def LoadPDB(filename: str):
    """ Read and save a protein structure from a PDB file
    Parameters
    ----------
    filename : string
        Name of the PDB file to be read
    Returns
    -------
    assembly : Assembly
        The full Assembly of the protein read from the PDB file, containing all chains with all residues
    """
    
    #AWESOME IMPLEMENTATION COMES HERE:
    
    # directly load full content in one list ('r' = read)
    with open(filename, 'r') as file:
        content = file.readlines()
        
    # define variables for everything we want to read, start with dummy/temporary names for the residue and chain
    assembly = Assembly()
    current_res = Residue(rname = 'dummy_res', rnum = None)
    current_chain = Chain(cname = 'dummy_chain')
    
    # go through each describing an atom in the PDB file
    for line in content:
        
        if line.startswith('ATOM'):
            
            # get all the non-empty values
            line_values = line.split(' ')
            line_values = [c for c in line_values if c != '']
            aname = line_values[2].strip() #13:17

            # we're only interested in those three backbone atoms
            if aname in ['N', 'CA', 'C']:

                # parse current line
                rname = line_values[3] #[17:21]
                rnum = line_values[5].strip() #23:27
                cname = line_values[4] #21
                x_coord = float(line_values[6].strip()) #30:38
                y_coord = float(line_values[7].strip()) #38:46
                z_coord = float(line_values[8].strip()) #46:54
                
                # check if there is a new residue
                if rnum != current_res.rnum or cname != current_chain.cname:
                    # we add the previous one to the current chain if all positions are valid
                    if current_res.n_pos and current_res.ca_pos and current_res.c_pos:
                        current_chain.residues.append(current_res)
                    # and generate the new residue
                    current_res = Residue(rname, rnum)
                
                # check if there is a new chain
                if cname != current_chain.cname:
                    # we add the previous one if it is not empty
                    if len(current_chain.residues) > 0:
                        assembly.chains.append(current_chain)
                    # and generate the new chain
                    current_chain = Chain(cname)
                
                # for the current residue, save the coordinates of the three backbone atoms as a Vec3 instance
                if aname == 'N':
                    current_res.n_pos = Vec3(x_coord, y_coord, z_coord)
                if aname == 'CA':
                    current_res.ca_pos = Vec3(x_coord, y_coord, z_coord)
                if aname == 'C':
                    current_res.c_pos = Vec3(x_coord, y_coord, z_coord)
                    
    # at the end, there still might be a complete residue and a complete chain that haven't
    # been added to the assembly
    if current_res.n_pos and current_res.ca_pos and current_res.c_pos:
        current_chain.residues.append(current_res)
    if len(current_chain.residues) > 0:
        assembly.chains.append(current_chain)
                
    return assembly

In [5]:
def get_aa(assembly: Assembly):
    ''' Get the three-letter amino acid sequence from an Assembly
    Warning: we assume here that our protein is a monomer (only one chain) and all the sequence is in the first chain
    Parameters
    ----------
    assembly : Assembly
        Assembly of a monomeric protein with the entire sequence in the first chain
    Returns
    -------
    aa_sequence : string
        The entire three-letter amino acid sequence from the first chain
    '''

    # initiate the value we want to compute
    aa_sequence = ''

    # extra only the sequence from the first chain
    first_chain = assembly.chains[0]

    for res in first_chain.residues:
        aa_sequence += res.rname + ' '

    return aa_sequence

In [6]:
def convert_sequence_to_one_letter(three_letter_sequence: str, delimiter: str = ' '):
    ''' Convert a three-letter code amino acid sequence, separated by the delimiter, to a one-letter code amino acid sequence
    Parameters
    ----------
    three_letter_sequence : string
        Three-letter code amino acid sequence to be converted, separated by the delimiter
    delimiter : string
        Delimiter for the three-letter code amino acid sequence
    Returns
    -------
    one_letter_sequence : string
        One-letter code amino acid sequence
    '''

    # define a dictionary mapping the three-letter codes to the one-letter codes
    mapping = {
    'ALA': 'A',
    'ARG': 'R',
    'ASN': 'N',
    'ASP': 'D',
    'CYS': 'C',
    'GLN': 'Q',
    'GLU': 'E',
    'GLY': 'G',
    'HIS': 'H',
    'ILE': 'I',
    'LEU': 'L',
    'LYS': 'K',
    'MET': 'M',
    'PHE': 'F',
    'PRO': 'P',
    'SER': 'S',
    'THR': 'T',
    'TRP': 'W',
    'TYR': 'Y',
    'VAL': 'V'
    }

    # intialize the value we want to compute
    one_letter_sequence = ''

    # remove the empty spaces at the beginning and end of the full sequence
    three_letter_sequence = three_letter_sequence.strip()

    # process and map each three letter code in the sequence, otherwise raise an error
    for three_letter_code in three_letter_sequence.split(delimiter):
        three_letter_code = three_letter_code.upper().strip()

        if three_letter_code in mapping:
            one_letter_sequence += mapping[three_letter_code]
        else:
            raise KeyError(f"Three-letter amino acid code '{three_letter_code}' not found.")

    return one_letter_sequence

In [7]:
def GetAlignedCaCoords(assembly1: Assembly, assembly2: Assembly, alignment1: str, alignment2: str):
    """ Given two assemblies and their alignment strings, return two arrays of the CA coordinates of aligned (not gaps) residues
    Warning: we assume here that our proteins are monomers (only one chain) and all the sequences are in the first chains
    Parameters
    ----------
    assembly1, assembly2 : Assembly
        Protein assemblies from which to get the CA coordinates of the aligned residues
    alignment1, alignment2 : string
        Aligned sequences (by Needleman-Wunsch algorithm for example) of the two assembly sequences
    Returns
    -------
    aligned_ca_coords1, aligned_ca_coords2 : numpy.array
        Numpy arrays of the CA coordinates of the aligned (not gaps) residues of the two input assemblies
    """

    # get the lists of all residues from the first chains in the assemblies
    residues1 = [residue for residue in assembly1.chains[0].residues]
    residues2 = [residue for residue in assembly2.chains[0].residues]

    # Initialize the variables we want to define
    aligned_ca_coords1 = []
    aligned_ca_coords2 = []
    res_index1 = 0
    res_index2 = 0

    # Iterate through the alignments
    for a1, a2 in zip(alignment1, alignment2):
        
        #check for the gaps and length
        if a1 != '-' and res_index1 < len(residues1):
            residue1 = residues1[res_index1]
            res_index1 += 1
        else:
            residue1 = None
            
        if a2 != '-' and res_index2 < len(residues2):
            residue2 = residues2[res_index2]
            res_index2 += 1
        else:
            residue2 = None
        
        # If both residues are not None, record CA coordinates
        if residue1 is not None and residue2 is not None:
            aligned_ca_coords1.append([residue1.ca_pos.x, residue1.ca_pos.y, residue1.ca_pos.z])
            aligned_ca_coords2.append([residue2.ca_pos.x, residue2.ca_pos.y, residue2.ca_pos.z])

    # Convert the lists to numpy arrays for easier manipulation.
    aligned_ca_coords1 = np.array(aligned_ca_coords1)
    aligned_ca_coords2 = np.array(aligned_ca_coords2)

    return aligned_ca_coords1, aligned_ca_coords2

In [8]:
def kabsch_superimpose(P: np.ndarray, Q: np.ndarray):
    """ Return the optimal rotation and translation matrices for superimposing the points P onto Q
    Parameters
    ----------
    P, Q : numpy ndarray
        Set of points: Nx3 matrix for N points in 3 dimensions to superimpose
    Returns
    -------
    rotation : numpy ndarray
        Optimal rotation matrix to align the set of point P and Q
    translation : numpy array
        Optimal translation vector to align the set of point P and Q
    """
    # this assertion is to make sure the number of points in P and Q are the same
    assert len(P) == len(Q)

    # np.mean calculates the average of the points in P and Q along the axis specified (axis=0 = per-column)
    # the centroid is the average position of all the points in each set
    centroid_P = np.mean(P, axis=0)
    centroid_Q = np.mean(Q, axis=0)

    # subtract the centroid from each set of points
    # this is done to move all points in both sets to the origin
    # which is a standard procedure before applying the Kabsch algorithm
    P_centered = P - centroid_P
    Q_centered = Q - centroid_Q

    # the dot product of the transpose of P and Q forms the covariance matrix
    # np.dot is a function that computes the dot product of two arrays
    H = np.dot(np.transpose(P_centered), Q_centered)

    # use Singular Value Decomposition (SVD) to decompose H
    # this is used to find the rotation matrix that will align the points in P with the points in Q
    U, S, Vt = np.linalg.svd(H)

    # To ensure a right-handed coordinate system, the determinant of U*Vt must be 1. If it's -1, we need to correct it
    d = (np.linalg.det(U) * np.linalg.det(Vt)) < 0.0

    if d:
        S[-1] = -S[-1]
        U[:, -1] = -U[:, -1]

    # Calculate the rotation matrix as the product of U and Vt
    rotation = np.dot(U, Vt)
    
    # The translation vector is calculated as the difference of centroids, transformed by the rotation matrix
    translation = centroid_Q - np.dot(centroid_P, rotation)

    return rotation, translation

def rmsd(V: np.ndarray, W: np.ndarray):
    """ Calculate Root-mean-square deviation (RMSD) from two sets of vectors V and W.
    Parameters
    ----------
    V, W : numpy ndarray
        (N,D) matrix, where N is points and D is dimension
    Returns
    -------
    rmsd : float
        Root-mean-square deviation
    """

    # initialize the variables
    N = len(V)
    D = len(V[0])
    
    if len(W[0]) != D or len(W) != N:
        raise ValueError(f"Error: The input vectors V and W do not have the same dimensions.")
    
    # compute the RMSD
    rmsd = 0.0

    for v, w in zip(V, W):
        rmsd += sum([(v[i] - w[i])**2.0 for i in range(D)])

    return np.sqrt(rmsd/N)

def transform(v: np.ndarray, rotation: np.ndarray, translation: np.array):
    """ Apply rotation and translation to the points v
    Parameters
    ----------
    v : numpy ndarray
        Matrix of points to be tranformed
    rotation : numpy ndarray
        Rotation matrix to transform v
    translation : numpy array
        Translation vector transform v
    Returns
    -------
    numpy : ndarray
        Tranformed matrix of v
    """
    return np.dot(v, rotation) + translation

In [11]:
# Example usage:

# These are random examples
P = np.random.rand(10,3)*100 
Q = np.random.rand(10,3)*100  

rotation, translation = kabsch_superimpose(P, Q)

# Apply the rotation and translation to P
P_tr = transform(P, rotation=rotation, translation=translation)

print('RMSD:', rmsd(P_tr, Q))

RMSD: 51.69238810933071


In [12]:
def get_superposition(P: np.ndarray, Q: np.ndarray):
    """ Combine all previous functions to get the RMSD of the superposition of P and Q
    Parameters
    ----------
    P, Q : numpy ndarray
        Set of points: Nx3 matrix for N points in 3 dimensions to superimpose
    Returns
    -------
    rmsd : float
        Root-mean-square deviation of the superimposed points P and Q
    """
    
    rotation, translation = kabsch_superimpose(P, Q)
    P = transform(P, rotation, translation)

    return rmsd(P, Q)

In [13]:
rmsd_value = get_superposition(P, Q)
print('RMSD:', rmsd_value)

RMSD: 51.69238810933071


Exercise: 

From the reference .pdb files for open (4ake) and closed (1ake) conformations, align their structures, get the matrices of the CA coordinates and print the RMSD.

In [20]:
# get the assembly objects from the PDB files
assembly_open = LoadPDB('4ake.pdb')
assembly_close = LoadPDB('1ake.pdb')

# get the three-letter code amino acid sequence from the assemblies and convert them to one-letter code sequences
open_three_letter = get_aa(assembly_open)
close_three_letter = get_aa(assembly_close)

open_one_letter = convert_sequence_to_one_letter(open_three_letter)
close_one_letter = convert_sequence_to_one_letter(close_three_letter)

# align the one-letter code sequences
aln_open, aln_close = Align(open_one_letter, close_one_letter)

print('Aligned sequence of the open structure:\n', aln_open)
print('Aligned sequence of the closed structure:\n', aln_close)

Aligned sequence of the open structure:
 MRIILLGAPGAGKGTQAQFIMEKYGIPQISTGDMLRAAVKSGSELGKQAKDIMDAGKLVTDELVIALVKERIAQEDCRNGFLLDGFPRTIPQADAMKEAGINVDYVLEFDVPDELIVDRIVGRRVHAPSGRVYHVKFNPPKVEGKDDVTGEELTTRKDDQEETVRKRLVEYHQMTAPLIGYYSKEAEAGNTKYAKVDGTKPVAEVRADLEKILG
Aligned sequence of the closed structure:
 MRIILLGAPGAGKGTQAQFIMEKYGIPQISTGDMLRAAVKSGSELGKQAKDIMDAGKLVTDELVIALVKERIAQEDCRNGFLLDGFPRTIPQADAMKEAGINVDYVLEFDVPDELIVDRIVGRRVHAPSGRVYHVKFNPPKVEGKDDVTGEELTTRKDDQEETVRKRLVEYHQMTAPLIGYYSKEAEAGNTKYAKVDGTKPVAEVRADLEKILG


In [21]:
# get the arrays of the CA coordinates of aligned residues
ca_coord_open, ca_coord_close = GetAlignedCaCoords(assembly_open, assembly_close, aln_open, aln_close)

# compute and print the RMSD value of the superposition
rmsd_value = get_superposition(ca_coord_open, ca_coord_close)
print('RMSD:', rmsd_value)

RMSD: 7.130700152775839


In [22]:
# you  can also print the matrices to see if it matches the coordinates of ca atoms in the pdb files
print(ca_coord_close)

[[26.091 52.849 39.889]
 [27.437 49.969 37.786]
 [24.961 47.988 35.671]
 [25.194 44.925 33.36 ]
 [22.428 44.503 30.712]
 [21.933 40.811 29.752]
 [19.604 39.512 26.973]
 [19.324 37.742 23.567]
 [20.108 39.651 20.294]
 [17.487 42.426 19.756]
 [15.96  42.201 23.284]
 [16.082 46.035 23.547]
 [19.303 46.427 25.595]
 [20.816 49.254 23.528]
 [17.421 51.124 23.373]
 [17.135 51.071 27.21 ]
 [20.821 51.825 27.722]
 [20.249 55.314 26.08 ]
 [17.474 56.05  28.712]
 [19.532 54.877 31.696]
 [22.441 56.903 30.479]
 [20.467 60.117 30.02 ]
 [18.497 59.71  33.252]
 [21.269 58.502 35.566]
 [24.253 60.217 33.938]
 [26.346 56.989 33.707]
 [28.231 56.198 30.472]
 [27.118 53.394 28.303]
 [30.122 51.105 27.381]
 [28.78  49.07  24.408]
 [31.156 46.587 22.987]
 [29.191 46.253 19.704]
 [29.487 50.042 19.273]
 [33.184 50.074 20.161]
 [33.94  47.195 17.763]
 [31.961 48.619 14.693]
 [33.778 51.895 15.373]
 [37.207 50.146 15.498]
 [36.426 48.279 12.266]
 [35.384 51.553 10.65 ]
 [38.507 53.491 11.905]
 [40.955 50.617 

Lecture 1: Introduction to alignments - exercise: align two homologous kinases (given as fasta files)

Lecture 2: Introduction to superpositions - exercise: get RMSD of two structures

<b>Lecture 3: Introduction to AlphaFold - exercise: get models from AlphaFold and classify them as open/close, do the same for models from Swiss-Model and AlphaFold-dropout</b>

Lecture 4: Ligand docking (DiffDock) - exercise: get docking from DiffDock colab, find interacting atoms and visualize docking

### 3. Lecture: Alphafold modelling, alignments, superposition

In [ ]:
# first start with getting the alphafold structures from colabfold (seq in previous ex) 
# and assign them as open/close
# then check if all 21 models from swiss-model and 40 models from alphafold-dropout also classify as close

In [23]:
def compare_with_assembly(model_assembly: Assembly, exp_assembly: Assembly):
    """ Compare a modeled assembly to the experimentally determined assembly
    Parameters
    ----------
    model_assembly, exp_assembly : Assembly
        Structure of a protein, saved as an Assembly object
    Returns
    -------
    rmsd_value : float
        Root-mean-square deviation of the superimposed of the model and experimental proteins
    """

    # use the same methods as in exercise 2 to get the RMSD of the two assemblies
    model_aa = get_aa(model_assembly)
    exp_aa = get_aa(exp_assembly)
    
    model_one_letter = convert_sequence_to_one_letter(model_aa)
    exp_one_letter = convert_sequence_to_one_letter(exp_aa)
    
    model_aln, exp_aln = Align(model_one_letter, exp_one_letter)
    
    model_aligned_ca_coords, exp_aligned_ca_coords = GetAlignedCaCoords(model_assembly, exp_assembly, model_aln, exp_aln)
    
    rmsd_value = get_superposition(model_aligned_ca_coords, exp_aligned_ca_coords)
    
    return rmsd_value

Classify the models obtained with standard AlphaFold to open or closed conformation using the experimentally determined structures

In [24]:
#these are the models they will get with standard alphafold (AF)
# also save the data in csv format for analysis in the fourth lecture
data = list()

# the os package allows you to easily navigate folders and files with the use of paths
import os

alphafold_model_dir = './alphafold_models_standard'
open_exp_structure = LoadPDB('4ake.pdb')
close_exp_structure = LoadPDB('1ake.pdb')

# get the list of all files in a directory
for f in os.listdir(alphafold_model_dir):

    # get the full path of the file f in the directory
    path = os.path.join(alphafold_model_dir, f)

    # get the assembly of the structure using the PDB file saved from AlphaFold
    af_assembly = LoadPDB(path)

    # compare the RMSD of the AF-structure with the experimentally-determined open and closed stuctures
    rmsd_open = compare_with_assembly(af_assembly, open_exp_structure) # How far away is Protein from open state
    rmsd_close = compare_with_assembly(af_assembly, close_exp_structure) # How far away is Protein from closed state
    
    print(f)
    print('RMSD_open:', rmsd_open)
    print('RMSD_close:', rmsd_close,'\n')

    if rmsd_open < rmsd_close: # closer to open state: Label = 0
        label = 0
        
    if rmsd_open > rmsd_close: # closer to closed state: label = 1
        label = 1

    data.append([f, rmsd_open, rmsd_close, label])

test_27f14_unrelaxed_rank_001_alphafold2_ptm_model_3_seed_000.pdb
RMSD_open: 7.222333381741465
RMSD_close: 0.778000547020972 

test_27f14_unrelaxed_rank_004_alphafold2_ptm_model_2_seed_000.pdb
RMSD_open: 5.606497270907733
RMSD_close: 2.039854829835252 

test_27f14_unrelaxed_rank_003_alphafold2_ptm_model_5_seed_000.pdb
RMSD_open: 7.284754137480984
RMSD_close: 0.776160871996097 

test_27f14_unrelaxed_rank_005_alphafold2_ptm_model_4_seed_000.pdb
RMSD_open: 7.0824691618723525
RMSD_close: 0.9261907145828077 

test_27f14_unrelaxed_rank_002_alphafold2_ptm_model_1_seed_000.pdb
RMSD_open: 6.054032248832644
RMSD_close: 1.8550082076994587 



Now classify the models obtained with AlphaFold dropout to open or closed conformations

In [25]:
# then check if alphafold-dropout does a better job



#TODO: calculate rmsd of models in "./alphafold_models_dropout" to experimental structures

# the os package allows you to easily navigate folders and files with the use of paths
import os

alphafold_model_dir = "./alphafold_models_dropout"
open_exp_structure = LoadPDB('4ake.pdb')
close_exp_structure = LoadPDB('1ake.pdb')

# get the list of all files in a directory
for f in os.listdir(alphafold_model_dir):

    # get the full path of the file f in the directory
    path = os.path.join(alphafold_model_dir, f)

    # get the assembly of the structure using the PDB file saved from AlphaFold
    af_assembly = LoadPDB(path)

    # compare the RMSD of the AF-structure with the experimentally-determined open and closed stuctures
    rmsd_open = compare_with_assembly(af_assembly, open_exp_structure) # How far away is Protein from open state
    rmsd_close = compare_with_assembly(af_assembly, close_exp_structure) # How far away is Protein from closed state
    
    print(f)
    print('RMSD_open:', rmsd_open)
    print('RMSD_close:', rmsd_close,'\n')

    if rmsd_open < rmsd_close: # closer to open state: Label = 0
        label = 0
        
    if rmsd_open > rmsd_close: # closer to closed state: label = 1
        label = 1

    data.append([f, rmsd_open, rmsd_close, label])




test_27f14_0_unrelaxed_rank_039_alphafold2_ptm_model_2_seed_003.pdb
RMSD_open: 4.770099326785428
RMSD_close: 2.677812763271696 

test_27f14_0_unrelaxed_rank_029_alphafold2_ptm_model_2_seed_005.pdb
RMSD_open: 5.61008960338677
RMSD_close: 1.920229129822461 

test_27f14_0_unrelaxed_rank_001_alphafold2_ptm_model_3_seed_007.pdb
RMSD_open: 7.2742157869092
RMSD_close: 0.8157055949847961 

test_27f14_0_unrelaxed_rank_035_alphafold2_ptm_model_4_seed_003.pdb
RMSD_open: 6.67057196924278
RMSD_close: 1.2081497148127518 

test_27f14_0_unrelaxed_rank_015_alphafold2_ptm_model_1_seed_002.pdb
RMSD_open: 5.741800398190161
RMSD_close: 2.0964445535937206 

test_27f14_0_unrelaxed_rank_036_alphafold2_ptm_model_4_seed_004.pdb
RMSD_open: 6.858423328334636
RMSD_close: 1.0451039974365202 

test_27f14_0_unrelaxed_rank_009_alphafold2_ptm_model_1_seed_005.pdb
RMSD_open: 6.084841328196615
RMSD_close: 1.8228666503514361 

test_27f14_0_unrelaxed_rank_002_alphafold2_ptm_model_3_seed_006.pdb
RMSD_open: 7.252437297511559

In [26]:
# then they will check how swiss-model is doing

#TODO: calculate rmsd of models in "./swiss_models" to experimental structures

# the os package allows you to easily navigate folders and files with the use of paths
import os

alphafold_model_dir = "./swiss_models"
open_exp_structure = LoadPDB('4ake.pdb')
close_exp_structure = LoadPDB('1ake.pdb')

# get the list of all files in a directory
for f in os.listdir(alphafold_model_dir):

    # get the full path of the file f in the directory
    path = os.path.join(alphafold_model_dir, f)

    # get the assembly of the structure using the PDB file saved from AlphaFold
    af_assembly = LoadPDB(path)

    # compare the RMSD of the AF-structure with the experimentally-determined open and closed stuctures
    rmsd_open = compare_with_assembly(af_assembly, open_exp_structure) # How far away is Protein from open state
    rmsd_close = compare_with_assembly(af_assembly, close_exp_structure) # How far away is Protein from closed state
    
    print(f)
    print('RMSD_open:', rmsd_open)
    print('RMSD_close:', rmsd_close,'\n')

    if rmsd_open < rmsd_close: # closer to open state: Label = 0
        label = 0
        
    if rmsd_open > rmsd_close: # closer to closed state: label = 1
        label = 1

    data.append([f, rmsd_open, rmsd_close, label])


model_09.pdb
RMSD_open: 7.457632384668484
RMSD_close: 0.9839113192898898 

model_21.pdb
RMSD_open: 5.962369229489549
RMSD_close: 3.64936521277075 

model_20.pdb
RMSD_open: 6.253750223410452
RMSD_close: 3.5694162818283193 

model_08.pdb
RMSD_open: 7.44378471068204
RMSD_close: 0.9997453061916688 

model_18.pdb
RMSD_open: 6.435332163548642
RMSD_close: 3.20034708709186 

model_19.pdb
RMSD_open: 6.173371313254284
RMSD_close: 3.3895320498911197 

model_14.pdb
RMSD_open: 5.234569482374888
RMSD_close: 4.811048292722425 

model_15.pdb
RMSD_open: 6.401471409976921
RMSD_close: 3.456491047767172 

model_01.pdb
RMSD_open: 7.240747404312423
RMSD_close: 0.4205283572155502 

model_17.pdb
RMSD_open: 6.2638229760471305
RMSD_close: 3.633737054697014 

model_03.pdb
RMSD_open: 6.9853361921947545
RMSD_close: 0.8049712535618541 

model_02.pdb
RMSD_open: 6.865318540805919
RMSD_close: 0.7999470553907604 

model_16.pdb
RMSD_open: 6.321605575647794
RMSD_close: 3.456564769612247 

model_12.pdb
RMSD_open: 2.091036

In [27]:
# save all the data to a csv file for the next lecture
np.savetxt('data.csv',
        data,
        delimiter =', ',
        header='filename, rmsd_open, rmsd_close, label',
        fmt ='% s')